In [33]:
# univariate multi-step encoder-decoder cnn-lstm
from math import sqrt
from numpy import split
from numpy import array
from pandas import read_csv
import pandas as pd
from sklearn.metrics import mean_squared_error
from keras.metrics import RootMeanSquaredError
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

In [137]:
#Lagged Data
X = read_csv('X_imp_cl_model_lag2.csv', index_col='DateTime', parse_dates=['DateTime'] )
X = X.rename(columns={'var1(t-24)': 'generation_t_24'})

y_lagged = read_csv('y_lag.csv', index_col='DateTime', parse_dates=['DateTime'])
y = pd.read_csv('generation_data.csv', parse_dates=['DateTime'], index_col=["DateTime"])
X_test = read_csv('X_imp_cl_test_lag.csv', index_col='DateTime', parse_dates=['DateTime'])
X_test = X_test.rename(columns={'var1(t-24)': 'generation_t_24'})


In [177]:
y.iloc[:24840,:]#[:'2021-10-31']

,Generation
DateTime,
2019-01-01 00:00:00,0.000000
2019-01-01 01:00:00,0.000000
2019-01-01 02:00:00,0.000008
2019-01-01 03:00:00,0.000000
2019-01-01 04:00:00,0.000008
...,...
2021-10-31 19:00:00,0.010345
2021-10-31 20:00:00,0.000000
2021-10-31 21:00:00,0.000000


In [140]:
merged = X.merge(y["Generation"], left_index=True, right_index=True, how='inner')
merged = merged.drop('generation_t_24', axis=1)

In [178]:
# split a univariate dataset into train/test sets
def split_dataset(data):
# split into standard weeks
    train, test = data[0:24840], data[24840:]
# restructure into windows of weekly data
    train = array(split(train, len(train)/24))
    test = array(split(test, len(test)/24))
    return train, test

In [56]:
# evaluate one or more weekly forecasts against expected values
def evaluate_forecasts(actual, predicted):
    scores = list()
    # calculate an RMSE score for each day
    for i in range(actual.shape[1]):
        # calculate mse
        mse = mean_squared_error(actual[:, i], predicted[:, i])
        # calculate rmse
        rmse = sqrt(mse)
        # store
        scores.append(rmse)
    # calculate overall RMSE
    s = 0
    for row in range(actual.shape[0]):
        for col in range(actual.shape[1]):
            s += (actual[row, col] - predicted[row, col])**2
    score = sqrt(s / (actual.shape[0] * actual.shape[1]))
    return score, scores

In [57]:
# summarize scores
def summarize_scores(name, score, scores):
    s_scores = ', '.join(['%.1f' % s for s in scores])
    print('%s: [%.3f] %s' % (name, score, s_scores))

In [58]:
# convert history into inputs and outputs
def to_supervised(train, n_input, n_out=24):
# flatten data
    data = train.reshape((train.shape[0]*train.shape[1], train.shape[2]))
    X, y = list(), list()
    in_start = 0
    # step over the entire history one time step at a time
    for _ in range(len(data)):
        # define the end of the input sequence
        in_end = in_start + n_input
        out_end = in_end + n_out
        # ensure we have enough data for this instance
        if out_end < len(data):
            x_input = data[in_start:in_end, 0]
            x_input = x_input.reshape((len(x_input), 1))
            X.append(x_input)
            y.append(data[in_end:out_end, 0])
        # move along one time step
        in_start += 1
    return array(X), array(y)

In [168]:
def build_model(train, n_input):
    # prepare data
    train_x, train_y = to_supervised(train, n_input)
    # define parameters
    verbose, epochs, batch_size = 1, 10, 32
    n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]
    # reshape output into [samples, timesteps, features]
    train_y = train_y.reshape((train_y.shape[0], train_y.shape[1], 1))
    # define model
    model = Sequential()
    model.add(Conv1D(filters=16, kernel_size=2, activation='relu', input_shape=(n_timesteps,n_features)))
    model.add(Conv1D(filters=16, kernel_size=2, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(RepeatVector(n_outputs))
    model.add(LSTM(50, activation='relu', return_sequences=True))
    model.add(TimeDistributed(Dense(25, activation='relu')))
    model.add(TimeDistributed(Dense(1)))
    model.compile(loss='mse', optimizer='adam', metrics=[RootMeanSquaredError()])
    # fit network
    model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, verbose=verbose)
    return model

In [162]:
# make a forecast
def forecast(model, history, n_input):
    # flatten data
    data = array(history)
    data = data.reshape((data.shape[0]*data.shape[1], data.shape[2]))
    # retrieve last observations for input data
    input_x = data[-n_input:, 0]
    # reshape into [1, n_input, 1]
    input_x = input_x.reshape((1, len(input_x), 1))
    # forecast the next week
    yhat = model.predict(input_x, verbose=0)
    # we only want the vector forecast
    yhat = yhat[0]
    return yhat

In [61]:
# evaluate a single model
def evaluate_model(train, test, n_input):
    # fit model
    model = build_model(train, n_input)
    # history is a list of weekly data
    history = [x for x in train]
    # walk-forward validation over each week
    predictions = list()
    for i in range(len(test)):
        # predict the week
        yhat_sequence = forecast(model, history, n_input)
        # store the predictions
        predictions.append(yhat_sequence)
        # get real observation and add to history for predicting the next week
        history.append(test[i, :])
    # evaluate predictions days for each week
    predictions = array(predictions)
    score, scores = evaluate_forecasts(test[:, :, 0], predictions)
    return score, score

In [183]:
merged = merged[['Generation', 'AirTemperature_t','wwcode_cluster_t',
                 'ComfortTemperature_t','RelativeHumidity_t',
                 'WindSpeed_t', 'WindDirection_t', 'is_holiday_t',
                 'WWCode_t',  'EffectiveCloudCover_t', 'Hour_t',
                 'DayGroup_t', 'Month_t',  'Day_t']]

In [184]:
n_input = 24

# split a univariate dataset into train/test sets
def split_dataset(data):
# split into standard weeks
    train, test = data[0:24840], data[24840:]#23352
# restructure into windows of weekly data
    train = array(split(train, len(train)/24))
    test = array(split(test, len(test)/24))
    return train, test

train, test = split_dataset(merged)

model = build_model(train, n_input)


history_test = [x for x in test]


# walk-forward validation over each week
predictions = list()
for i in range(len(test)):
    # predict the week
    yhat_sequence = forecast(model, history_test, n_input)
    predictions.append(yhat_sequence)
    # get real observation and add to history for predicting the next week
    history_test.append(test[i, :])
    # evaluate predictions days for each week
predictions = array(predictions)    

Epoch 1/10
775/775 [==============================] - 11s 13ms/step - loss: 13453.2207 - root_mean_squared_error: 115.9881
Epoch 2/10
775/775 [==============================] - 10s 13ms/step - loss: 9961.5791 - root_mean_squared_error: 99.8077
Epoch 3/10
775/775 [==============================] - 10s 13ms/step - loss: 6001.4487 - root_mean_squared_error: 77.4690
Epoch 4/10
775/775 [==============================] - 10s 13ms/step - loss: 6568.5962 - root_mean_squared_error: 81.0469
Epoch 5/10
775/775 [==============================] - 10s 13ms/step - loss: 4604.5537 - root_mean_squared_error: 67.8569
Epoch 6/10
775/775 [==============================] - 10s 13ms/step - loss: 5657.2642 - root_mean_squared_error: 75.2148
Epoch 7/10
775/775 [==============================] - 10s 13ms/step - loss: 7248.5918 - root_mean_squared_error: 85.1387
Epoch 8/10
775/775 [==============================] - 10s 13ms/step - loss: 5692.3384 - root_mean_squared_error: 75.4476
Epoch 9/10
775/775 [==========

In [185]:
print(np.array(predictions.flat).max())
print(np.array(predictions.flat).min())

377.56927
-60.58003


In [187]:
total_capacity = 523.005001

y_test = np.array([total_capacity*0.98 if i > total_capacity else 0 if i < 0 else i for i in np.array(predictions.flat)])

mean_squared_error(test.reshape(-1, 14)[:,0], y_test, squared=False)

49.838869341290014